In [1]:
import pandas as pd
import numpy as np

cell_size = 0.0083333333333333
NODATA_value = -9999
start_x = -180
start_y = 90

filepath_base = "gpw-v4-population-count-rev11_2020_30_sec_asc/gpw_v4_population_count_rev11_2020_30_sec_"
grid = None
for col in range(2):
    grid_line = None
    for row in range(4):
        filepath = filepath_base + str(col * 4 + row + 1) + ".asc"
        # grid_part = np.loadtxt(filepath, skiprows=6)
        grid_part = pd.read_csv(filepath,header=None,na_filter=False,delim_whitespace=True, skiprows=6).to_numpy()
        if grid_line is None:
            grid_line = grid_part
        else:
            grid_line = np.concatenate((grid_line, grid_part), axis=1)
    if grid is None:
        grid = grid_line
    else :
        grid = np.concatenate((grid, grid_line), axis=0)

In [2]:
grid.shape

(21600, 43200)

In [8]:
print (np.where(grid != NODATA_value))

(array([  764,   764,   764, ..., 17518, 17518, 17518]), array([17677, 17678, 17679, ..., 13527, 13528, 13529]))


In [2]:
# !pip install shapely

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
from shapely.geometry import shape, Point
from shapely.prepared import prep
import numpy as np
import json
# cell_size = 0.0083333333333333
# NODATA_value = -9999
with open("1.geojson") as gj:
    gjson = json.load(gj)

polygon = shape(gjson['features'][0]['geometry'])
(latmin, lonmin, latmax, lonmax) =((x // cell_size * cell_size) for x in polygon.bounds)
prep_polygon = prep(polygon)
points = []
for lat in np.arange(latmin, latmax, cell_size):
    for lon in np.arange(lonmin, lonmax, cell_size):
        points.append(Point((round(lat,4), round(lon,4))))
valid_points = []
valid_points.extend(filter(prep_polygon.contains, points))
# print(valid_points)

In [6]:
sum = 0
for point in  valid_points:
    lon, lat = point.x, point.y
    local_sum = grid[int((start_y - lat) // cell_size)][int((lon - start_x) // cell_size)]
    if local_sum != NODATA_value:
        sum += int(local_sum)
print(sum)

28109088


In [57]:
# !pip install geopandas
import geopandas as gpd
import json
df_places = gpd.read_file('jsonp.json')
df_places2 = gpd.read_file('jsonp1.json')
with open("jsonp.json") as gj:
    gjson = json.load(gj)
print(df_places2['name'])
print(df_places['name'])
# df_places.tail()
# # print(df_places.to_json())
# for index, row in df_places.iterrows():
#     print(row)

0       北京
1       天津
2       河北
3       山西
4      内蒙古
5       辽宁
6       吉林
7      黑龙江
8       上海
9       江苏
10      浙江
11      安徽
12      福建
13      江西
14      山东
15      河南
16      湖北
17      湖南
18      广东
19      广西
20      海南
21      重庆
22      四川
23      贵州
24      云南
25      西藏
26      陕西
27      甘肃
28      青海
29      宁夏
30      新疆
31      台湾
32      香港
33      澳门
34    南海诸岛
35    南海诸岛
Name: name, dtype: object
0          北京市
1          天津市
2          河北省
3          山西省
4       内蒙古自治区
5          辽宁省
6          吉林省
7         黑龙江省
8          上海市
9          江苏省
10         浙江省
11         安徽省
12         福建省
13         江西省
14         山东省
15         河南省
16         湖北省
17         湖南省
18         广东省
19     广西壮族自治区
20         海南省
21         重庆市
22         四川省
23         贵州省
24         云南省
25       西藏自治区
26         陕西省
27         甘肃省
28         青海省
29     宁夏回族自治区
30    新疆维吾尔自治区
31         台湾省
32          香港
33          澳门
34            
Name: name, dtype: object


In [46]:
!pip install asyncio

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 101 kB 397 kB/s ta 0:00:011
